In [1]:
import pandas as pd
import json

In [2]:
folder = '.\\input_files\\'

file = '341567805_uh-cleveland-medical-center_standardcharges.json'

In [3]:
file_path = folder + file
with open(file_path,'r') as f:
    jf = json.load(f)
jf = jf[0]

In [4]:
df = pd.DataFrame(jf['item'])

In [5]:
df = df.assign(Associated_Codes=df['Associated_Codes'].str.split(',')).explode('Associated_Codes')

In [6]:
df.rename(columns={
    'payer': 'payer_name',
    'Associated_Codes': 'code',
    'iobSelection': 'setting',
    'Payer_Allowed_Amount': 'standard_charge',
}, inplace=True)

In [7]:
df_payer = df.copy()
df_payer = df_payer[['payer_name', 'description', 'code', 'standard_charge', 'setting']]
df_payer['payer_category'] = 'payer'

In [8]:
df_rates = df.copy()
df_rates = df_rates[['description', 'code', 'setting', 'Avg_Gross_Charge', 'Cash_Discount', 'Deidentified_Min_Allowed', 'DeIdentified_Max_Allowed']]

In [9]:
cols = df_rates.columns.tolist()
id_vars = cols[:3]
value_vars = cols[3:]

df_rates = pd.melt(df_rates, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [10]:
mapping = {
    'Avg_Gross_Charge': 'gross',
    'Cash_Discount': 'cash',
    'Deidentified_Min_Allowed': 'min',
    'DeIdentified_Max_Allowed':'max',
    }

df_rates['payer_category'] = df_rates['payer_name'].map(mapping)

In [11]:
df = pd.concat([df_payer, df_rates])

In [12]:
df.reset_index(drop=True, inplace=True)

In [13]:
df['setting'] = df['setting'].str.lower()

In [14]:
df.loc[df['code'].str.match(r'^[A-Z][0-9]{4}|[0-9]{5}|[0-9]{4}[A-Z]$'), 'hcpcs_cpt'] = df['code']
df.loc[df['code'].str.match(r'[0-9]{3}'), 'ms_drg'] = df['code']
df.loc[df['code'].str.len() == 4, 'rev_code'] = df['code']

In [15]:
ccns = {'340714535': '360002',
 '340827442': '360041',
 '461382538': '360078',
 '341425870': '360098',
 '341260978': '360123',
 '341567805': '363302',
 '340714612': '360145',
 '340816492': '360192',
 '264827222': '360359',
 '340714461': '361307',
 '340714550': '361308',
 '371848577': '360367'}

ein = file.split('_')[0]
id = ccns[ein]

df['hospital_id'] = id

In [21]:
df.loc[df['code'].str.len() == 2]

,payer_name,description,code,standard_charge,setting,payer_category,hcpcs_cpt,ms_drg,rev_code,hospital_id,apr_drg
2892,Buckeye Community Health Plan Medicaid,ALLOGENEIC BONE MARROW TRANSPLANT,71,131899.3500,inpatient,payer,NaN,NaN,NaN,363302,000-71
2893,Buckeye Community Health Plan Medicaid,ALLOGENEIC BONE MARROW TRANSPLANT,72,131899.3500,inpatient,payer,NaN,NaN,NaN,363302,000-72
2894,Buckeye Community Health Plan Medicaid,ALLOGENEIC BONE MARROW TRANSPLANT,73,131899.3500,inpatient,payer,NaN,NaN,NaN,363302,000-73
2895,Buckeye Community Health Plan Medicaid,ALLOGENEIC BONE MARROW TRANSPLANT,74,131899.3500,inpatient,payer,NaN,NaN,NaN,363302,000-74
2965,Buckeye Community Health Plan Medicaid,AUTOLOGOUS BONE MARROW TRANSPLANT OR T-CELL IM...,81,70372.7200,inpatient,payer,NaN,NaN,NaN,363302,000-81
...,...,...,...,...,...,...,...,...,...,...,...
98373,DeIdentified_Max_Allowed,AUTOLOGOUS BONE MARROW TRANSPLANT OR T-CELL IM...,84,70372.7200,inpatient,max,NaN,NaN,NaN,363302,000-84
98784,DeIdentified_Max_Allowed,EXTRACORPOREAL MEMBRANE OXYGENATION (ECMO),91,99220.6700,inpatient,max,NaN,NaN,NaN,363302,000-91
98785,DeIdentified_Max_Allowed,EXTRACORPOREAL MEMBRANE OXYGENATION (ECMO),92,99220.6700,inpatient,max,NaN,NaN,NaN,363302,000-92
98786,DeIdentified_Max_Allowed,EXTRACORPOREAL MEMBRANE OXYGENATION (ECMO),93,99220.6700,inpatient,max,NaN,NaN,NaN,363302,000-93


In [20]:
df.loc[df['code'].str.len() == 2, 'apr_drg'] = '000-' + df['code'] 

In [16]:
# df.to_csv('.\\output_files\\' + ein + '_' id + '.csv', index=False)